<a href="https://colab.research.google.com/github/johyunkang/python_stat/blob/main/chapter11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 11 통계적 가설 검정

## 11.1 통계적 가설검정

In [ ]:
from scipy import stats
import pandas as pd

df = pd.read_csv('감자튀김.csv')
sample = np.array(df['무게'])


rv = stats.norm(130, np.sqrt(9/14)) # 감튀 평균:130, 분산:9, 표본: 14개
print(rv.isf(0.95)) # isf (Inverse Survival Function) : 임곗값 산출
> -1.645

potato_mean = np.mean(sample)
print(potato_mean)
> 128.451

Z = (potato_mean - 130) / np.sqrt(9/14) # 검정통계량 값 Z 를 구함
print(Z)
> -1.932

# p-value 는 누적분포함수(cdf)를 이용해 산출 가능
p-val = rv.cdf(Z)
print(p-val)
> 0.027

# p-value 가 유의수준(a) 0.05 보다 작으므로 귀무가설을 기각한다.


$Z = \dfrac{(\bar{X} - 130)} { \sqrt{\dfrac{9} {14}}}$

p-value 를 기준으로 수행하는 가설검정의 흐름

## 11.2 기본적인 가설검정

### 11.2.1 정규분포의 모평균에 대한 검정 : 모분산을 알고 있는 경

모평균에 대한 검정이란 **모평균이 어떤 값 $\mu_0$이 아니라고 주장**하기 위한 검정


검정통계량으로 Z를 사용 (양측검정)

$Z = \dfrac {(\bar{X} - \mu_0)} {\sqrt{\dfrac {\sigma^2} {n}}} $

- 귀무가설을 기각 : $Z < z_{1 - \alpha / 2} \quad or \quad z_{a/2} < Z$
- 귀무가설을 채택 : $z_{1-\alpha/2} \leq  Z \leq z_{\alpha/2}$


위 식을 아래 파이썬으로 구현

In [7]:
def potato_mean_test(sample, mean0, p_var, alpha=0.05) :
    import numpy as np
    from scipy import stats

    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1 - alpha)

    z = (s_mean - mean0) / np.sqrt(p_var / n)

    if interval[0] <= z <= interval[1] :
        print(f'유의수준 {alpha:.3f}에서 귀무가설을 채택')
    else :
        print(f'유의수준 {alpha:.3f}에서 귀무가설을 기각')
    
    if z < 0 :
        p_value = rv.cdf(z) * 2 # 양측검정이라 * 2 해줌
    else :
        p_value = (1- rv.cdf(z)) * 2 # 양측검정이라 * 2 해줌

    print(f'p-value는 {p_value:.3f} 입니다.')

In [9]:
potato_sample = [122.02, 131.73, 130.6, 131.82, 132.05, 126.12, 124.43,
          132.89, 122.79, 129.95, 126.14, 134.45, 127.64, 125.68]

mu = 130
var = 9
potato_mean_test(potato_sample, mu, var)

유의수준 0.050에서 귀무가설을 채택
p-value는 0.053 입니다.


### 11.2.2 정규분포의 모분산에 대한 검정

모분산에 대한 검정은 **모분산이 어떤 값 $\sigma_0^2$ 이 아닌 것을 주장하기 위한 검정**입니다.

$N(\mu, \sigma^2)$ 이라고 함. 모분산 $\sigma^2$에 관한 유의수준 $\alpha$의 양측검정

- 귀무가설 : $\sigma^2 = \sigma_0^2$
- 대림가설 : $\sigma^2 \neq \sigma_0^2$

검정통계량으로 $Y = \dfrac {(n-1)s^2} {\sigma_0^2}$ 을 사용

- 귀무가설을 채택 : $X_{1-\alpha/2}^2(n-1)  \leq Y \leq X_{\alpha/2}^2 (n-1)$
- 귀무가설을 기각 : $Y \lt X_{1-\alpha/2}^2(n-1) \quad or \quad X_{\alpha/2}^2 (n-1) \lt Y$

In [17]:
def potato_var_test(sample, var0, alpha=0.05) :
    import numpy as np
    from scipy import stats

    u_var = np.var(sample, ddof=1) # ddof=1 : 불편분산 (n-1 사용), ddof=0 : 표본분산 (n 사용)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1 - alpha)

    y = (n - 1) * u_var / var0

    if interval[0] <= y <= interval[1] :
        print(f'유의수준 {alpha:.3f}에서 귀무가설을 채택합니다')
    else :
        print(f'유의수준 {alpha:.3f}에서 귀무가설을 기각합니다')
    
    if y < rv.isf(0.5) : # isf (Inverse Survival Function)
        p = rv.cdf(y) * 2 # 양측검정이라 * 2 
    else :
        p = (1 - rv.cdf(y)) * 2 # 양측검정이라 * 2
    
    print(f'p-value는 {p:.3f} 입니다.')
    print('Interval 값은 :', interval)
    print('검정통계량 Y : ', y)

In [18]:
potato_var_test(potato_sample, var)

유의수준 0.050에서 귀무가설을 채택합니다
p-value는 0.085 입니다.
Interval 값은 : (5.008750511810331, 24.735604884931547)
검정통계량 Y :  22.921810317460263


### 11.2.3 정규분포의 모평균에 대한 검정 : 모분산을 모르는 경우

p.311